In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Configure Matplotlib

Choose a matplotlib style sheet to match your browser theme (dark or light)

Increase the figure size to match the notebook width.

Set the graph colour variables.

In [6]:
#style_sheet = 'dark_background' # dark theme
style_sheet = 'default'         # light theme
plt.style.use(style_sheet)

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200

dsc_colour = 'green'
wnd_colour = 'magenta'
anl_colour = 'red'

# Time Conversion functions

Let's define some helper functions to convert some time formats into human readable time

In [7]:
def time_from_seconds(seconds):
  seconds = seconds % (24 * 3600)
  hour = seconds // 3600
  seconds %= 3600
  minutes = seconds // 60
  seconds %= 60

  return "%d:%02d:%02d" % (hour, minutes, seconds)

def time_from_doy(doy):
  part, whole = np.modf(doy)
  seconds = part * 86400

  return time_from_seconds(seconds)

In [8]:
print("3600 seconds: %s" %(time_from_seconds(3600)))
print("0.04167 days: %s" %(time_from_doy(0.04167)))

3600 seconds: 1:00:00
0.04167 days: 1:00:00


# DSCOVR data

The DSCOVR  data is provided in the challenge resources. We will use 2022 data for our training and validation sets and 2023 data for a test set.
In machine learning we call the data we have, *X*. and teach (train) a model to learn a mapping of *X* to *Y* where *Y* is what we want the model to output.

Before we move on to *Y* let's have a look at *X* ...

First we need to download the data that we need from the UH/IfA data transfer node [spaceapps repo](https://dtn-itc.ifa.hawaii.edu/spaceapps/DSCOVR.tgz)

In [9]:
!wget https://opensource.gsfc.nasa.gov/spaceappschallenge/dsc_fc_summed_spectra_2022_v01.zip
!wget https://opensource.gsfc.nasa.gov/spaceappschallenge/dsc_fc_summed_spectra_2023_v01.zip
!mkdir -p ./data/dscovr
!(cd ./data/dscovr;unzip ../../dsc_fc_summed_spectra_2022_v01.zip; unzip ../../dsc_fc_summed_spectra_2023_v01.zip)


--2023-10-08 20:21:39--  https://opensource.gsfc.nasa.gov/spaceappschallenge/dsc_fc_summed_spectra_2022_v01.zip
Resolving opensource.gsfc.nasa.gov (opensource.gsfc.nasa.gov)... 129.164.181.182, 2001:4d0:2310:153::6
Connecting to opensource.gsfc.nasa.gov (opensource.gsfc.nasa.gov)|129.164.181.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57868570 (55M) [application/zip]
Saving to: ‘dsc_fc_summed_spectra_2022_v01.zip’

dsc_fc_summed_spect 100%[===================>]  55.19M  2.31MB/s    in 27s     

2023-10-08 20:22:06 (2.04 MB/s) - ‘dsc_fc_summed_spectra_2022_v01.zip’ saved [57868570/57868570]

--2023-10-08 20:22:06--  https://opensource.gsfc.nasa.gov/spaceappschallenge/dsc_fc_summed_spectra_2023_v01.zip
Resolving opensource.gsfc.nasa.gov (opensource.gsfc.nasa.gov)... 129.164.181.182, 2001:4d0:2310:153::6
Connecting to opensource.gsfc.nasa.gov (opensource.gsfc.nasa.gov)|129.164.181.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

In [10]:
!ls ./data/dscovr

dsc_fc_summed_spectra_2022_v01.csv  dsc_fc_summed_spectra_2023_v01.csv


# DSCOVR data
The DSCOVR data we need for training our model is now stored locally in ./data.
Let's take a look at the data we just downloaded.

In [ ]:
dscovr_df = pd.read_csv("./data/dscovr/dsc_fc_summed_spectra_2022_v01.csv", delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)

In [ ]:
dscovr_df.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,2022-01-01 00:00:00,-6.17170,1.124830,-4.90228,38.0314,0.231726,46.0427,44.9743,55.9143,43.7069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 00:01:00,-6.28883,1.233130,-4.79001,38.3868,0.231726,45.5257,46.2587,55.1428,43.2768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01 00:02:00,-6.11811,0.871923,-5.12830,37.5636,0.231726,45.1955,46.8222,55.7484,42.7894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01 00:03:00,-6.28704,1.249870,-4.76640,38.1094,0.242084,46.7083,47.1713,53.5380,42.1558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01 00:04:00,-6.42125,1.171560,-4.53230,37.5893,0.231726,47.4888,45.3234,54.5404,44.2773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dscovr_df.describe()

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
count,524450.000000,524450.000000,524450.000000,511816.000000,511811.000000,511809.000000,511809.000000,511815.000000,511815.000000,511814.000000,...,20115.000000,11191.000000,8416.000000,3971.000000,3859.000000,213.000000,139.000000,44.000000,40.000000,35.000000
mean,0.098939,-0.219225,0.060599,58.218162,9.514369,63.246343,56.563212,74.062121,65.428809,94.261894,...,384.711976,332.966637,388.589832,326.915073,294.150454,390.130451,384.123885,376.326977,403.068593,367.789749
std,3.943064,4.535931,3.667523,38.506281,17.031107,44.696797,45.047853,60.528255,82.509457,114.361904,...,40.774888,48.215582,38.280578,57.801942,85.771695,52.754546,51.583450,69.893480,124.156716,143.007012
min,-16.712300,-26.376500,-23.060000,0.231726,0.231726,0.231726,0.231726,0.231726,0.231726,0.231726,...,214.319000,188.544000,211.826000,205.870000,108.950000,202.247000,112.290000,187.778000,0.231726,89.996200
25%,-3.000140,-3.305092,-1.967988,28.843000,0.231726,32.352500,27.916000,37.301550,25.640400,36.973425,...,365.230000,302.532500,367.203750,274.956000,211.853000,359.223000,352.946500,354.073000,366.691750,241.622000
50%,0.212469,-0.288369,0.034746,51.989300,0.386813,55.570600,46.977600,61.355300,45.470200,68.838050,...,383.495000,326.713000,393.807500,323.581000,309.980000,389.128000,395.347000,383.129500,427.067000,403.663000
75%,3.164427,3.017987,2.039108,84.737900,15.812800,90.033900,80.592700,100.403000,84.142500,114.472000,...,404.069000,363.782500,409.080750,377.611000,367.034500,416.967000,410.484000,411.971250,471.866000,449.118500
max,19.725300,22.834700,27.905900,415.389000,385.676000,493.748000,746.807000,1136.670000,1562.550000,1804.560000,...,662.013000,735.132000,772.122000,521.745000,541.136000,637.731000,568.053000,528.470000,655.892000,646.111000


Take a look at the data statistics. There are not always the same number of rows for each column so we need to watch for missing data. The index is the timestamp for each row. The values in columns 1-3 are the magnetic field vector. The remaining columns contain the solar wind values.

# Daily Space Weather Kp indices

Next let's download the rest of our training data. This will be the *Y* that we want to predict. It is an integer value between 0 and 9 that describes the strength of the predicted geomagnetic field and relates to the likelihood of a geomagnetic storm.

We have staged the data for 2022 and 2023 that we obtained from the [NOAA web site](ftp:/ftp.ngdc.noaa.gov/STP/swpc_products/daily_reports/space_weather_indices) on our UH/IfA data transfer node [spaceapps data repo](http://dtn-itc.ifa.hawaii.edu/spaceapps/SWID.tgz)

In [11]:
!wget http://dtn-itc.ifa.hawaii.edu/spaceapps/SWID.tgz
!tar xzvf SWID.tgz


--2023-10-08 20:22:28--  http://dtn-itc.ifa.hawaii.edu/spaceapps/SWID.tgz
Resolving dtn-itc.ifa.hawaii.edu (dtn-itc.ifa.hawaii.edu)... 128.171.123.136
Connecting to dtn-itc.ifa.hawaii.edu (dtn-itc.ifa.hawaii.edu)|128.171.123.136|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 127530 (125K) [application/x-gzip]
Saving to: ‘SWID.tgz’

SWID.tgz            100%[===================>] 124.54K   104KB/s    in 1.2s    

2023-10-08 20:22:30 (104 KB/s) - ‘SWID.tgz’ saved [127530/127530]

./data/swpc/
./data/swpc/2022/
./data/swpc/2022/04/
./data/swpc/2022/04/20220401dayind.txt
./data/swpc/2022/04/20220402dayind.txt
./data/swpc/2022/04/20220403dayind.txt
./data/swpc/2022/04/20220404dayind.txt
./data/swpc/2022/04/20220405dayind.txt
./data/swpc/2022/04/20220406dayind.txt
./data/swpc/2022/04/20220407dayind.txt
./data/swpc/2022/04/20220408dayind.txt
./data/swpc/2022/04/20220409dayind.txt
./data/swpc/2022/04/20220410dayind.txt
./data/swpc/2022/04/20220411dayind.txt
./data/swpc

In [12]:
!tail -9 ./data/swpc/2023/10/20231007dayind.txt

# ----- Fredericksburg -----           --------- Boulder ---------
# A        K-indices                   A        K-indices
#     03-06-09-12-15-18-21-24              03-06-09-12-15-18-21-24
 -1    1  2 -1 -1 -1 -1 -1 -1          4    1  2  1  0  1  1  1  2 
#        High Latitude                                  Estimated
# --------- College ---------      -------------------- Planetary --------------------
# A       K-indices                A                    K-indices
#     03-06-09-12-15-18-21-24            03  - 06  - 09  - 12  - 15  - 18  - 21  - 24
  2    1  1  1  0  0  0  1  0      5    1.33  2.00  1.33  0.67  0.33  0.33  1.67  1.67 


We will need to parse each of these files, grabbing the last line and the last 8 floating point values which will be the daily planetary K-indices that we want to predict.

In [ ]:
!tail -1 ./data/swpc/2023/10/20231007dayind.txt | awk '{print $11, $12, $13, $14, $15, $16, $17, $18}'

1.33 2.00 1.33 0.67 0.33 0.33 1.67 1.67


We provide a useful shell script to do this for every file for 2022 and save the output in a single csv file. One file for 2022 and one file for 2023.

In [ ]:
!wget http://dtn-itc.ifa.hawaii.edu/spaceapps/parse_kp_indices.sh
!bash ./parse_kp_indices.sh

--2023-10-08 20:11:28--  http://dtn-itc.ifa.hawaii.edu/spaceapps/parse_kp_indices.sh
Resolving dtn-itc.ifa.hawaii.edu (dtn-itc.ifa.hawaii.edu)... 128.171.123.136
Connecting to dtn-itc.ifa.hawaii.edu (dtn-itc.ifa.hawaii.edu)|128.171.123.136|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462 [text/x-sh]
Saving to: ‘parse_kp_indices.sh.1’

parse_kp_indices.sh 100%[===================>]     462  --.-KB/s    in 0s      

2023-10-08 20:11:28 (56.9 MB/s) - ‘parse_kp_indices.sh.1’ saved [462/462]



In [ ]:
kp_df = pd.read_csv("./data/swpc/kpindices-2022.csv", delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)

In [ ]:
kp_df.head()

,0,1,2,3,4,5,6,7,8
0,date,3.0,6.0,9.0,12.0,15.0,18.0,21.0,24.0
1,20221122,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2,20221129,4.0,3.0,4.0,4.0,4.0,5.0,4.0,3.0
3,20221107,1.0,1.0,2.0,3.0,4.0,5.0,4.0,3.0
4,20221105,3.0,3.0,2.0,3.0,2.0,2.0,1.0,1.0


In [ ]:
kp_df.describe()

,1,2,3,4,5,6,7,8
count,348.000000,336.000000,343.000000,355.000000,352.000000,342.00000,341.000000,348.000000
mean,2.468391,2.389881,2.215743,2.219718,2.196023,2.25731,2.255132,2.390805
std,1.172023,1.201896,1.129140,1.140995,1.289310,1.41769,1.532782,1.651870
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000
25%,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000
max,6.000000,7.000000,9.000000,12.000000,15.000000,18.00000,21.000000,24.000000


As you can see, the counts don't agree for every column so we have some data cleanup yet to do.

However, we have the training data in place now. We are going to train a model to learn to predict the daily Kp indices using the DSCOVR data from the previous few days. Take a minute to think about what model architectures might be useful for this then open the dscovrmatic-dataprep notebook to continue.